In [ ]:
%reset -f

In [1]:
%matplotlib inline
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tqdm import tqdm
import numpy as np

import os

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
!cd ../input && ls

genre-images-wb  images-450-257  mnistkeras  textfiles


In [3]:
GENRE_DIR = '../input/images-450-257/images_450_257/genres_images_450_257/'
# GENRE_DIR = '../input/genre-images-wb/images_wb/genre_images_wb/'
GENRE_LIST = ('blues', 'classical', 'country', 'disco', 
              'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock')
NUM_OF_GENRES = 10
GENRE_SONGS = 100

In [4]:
from PIL import Image
import imageio

## Read Data

In [5]:
img_shape = (257, 450, 1)

In [6]:
# img_shape = (513, 800, 1)

In [7]:
x = np.ndarray((NUM_OF_GENRES * GENRE_SONGS, *img_shape))
y = []

In [8]:
def show(image):
    plt.imshow(image)

In [9]:
pos = 0
for genre_idx, genre in enumerate(GENRE_LIST):
    print(genre, '...')
    for root, subdirs, files in os.walk(GENRE_DIR + genre):
        for image_file in tqdm(files):
            img = imageio.imread(root + '/' + image_file)
            try:
                if len(img.shape) == 2:
                    tmp = np.array(img).reshape(img_shape)
                else:
                    tmp = np.array(img[:, :, :img_shape[-1]])
            except Exception as e:
                print(e)
                print(img.shape)
            else:
                x[pos] = tmp
                y.append(genre_idx)
                pos += 1
#             plt.subplot(121)
#             show(tmp)
            
#             plt.subplot(122)
#             show(x[pos][:, :, :-1])
#             print((tmp == x[pos]).all())
            
#             break
        break
#     break

 16%|█▌        | 16/100 [00:00<00:00, 152.81it/s]

blues ...


 20%|██        | 20/100 [00:00<00:00, 195.04it/s]

classical ...


 19%|█▉        | 19/100 [00:00<00:00, 189.40it/s]

...

 21%|██        | 21/100 [00:00<00:00, 202.15it/s]

reggae ...


 20%|██        | 20/100 [00:00<00:00, 199.73it/s]

rock ...


100%|██████████| 100/100 [00:00<00:00, 178.53it/s]


In [10]:
x = x[:pos]

In [11]:
y = np.array(y)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=53, stratify=y)

In [13]:
batch_size = 5
num_classes = NUM_OF_GENRES
epochs = 12

# input image dimensions
img_rows, img_cols, components = img_shape

# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

In [14]:
K.image_data_format()

'channels_last'

In [15]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], components, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], components, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, components)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, components)
    input_shape = (img_rows, img_cols, components)

In [16]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (800, 257, 450, 1)
800 train samples
200 test samples


In [17]:
x.shape

(1000, 257, 450, 1)

In [18]:
batch_size = 10

In [ ]:
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3), strides=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
# model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
print('ok')

In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size=2, strides=1, activation='relu', input_shape=input_shape))
model.add(Conv2D(16, kernel_size=2, strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(32, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(32, kernel_size=4, strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_OF_GENRES, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# Test loss: 2.121414485919786
# Test accuracy: 0.495
model = Sequential()
model.add(Conv2D(8, kernel_size=2, activation='relu', input_shape=input_shape))
model.add(Conv2D(8, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(16, kernel_size=4, activation='relu'))
model.add(Conv2D(16, kernel_size=4, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
# model.add(Dropout(0.1))
# model.add(Dense(32, activation='relu'))
model.add(Dense(NUM_OF_GENRES, activation='softmax'))

model.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=["accuracy"])

In [35]:
# Test loss: 2.121414485919786
# Test accuracy: 0.495
model = Sequential()
model.add(Conv2D(4, kernel_size=2, activation='relu', input_shape=input_shape))
model.add(Conv2D(4, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.05))
model.add(Conv2D(8, kernel_size=2, activation='relu'))
model.add(Conv2D(8, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.05))
model.add(Dense(32, activation='relu'))
model.add(Dense(NUM_OF_GENRES, activation='softmax'))
model.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=["accuracy"])

In [36]:
batch_size = 5

In [37]:
epochs = 24

In [40]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 800 samples, validate on 200 samples
Epoch 1/24
800/800 [==============================] - 4s 5ms/step - loss: 0.0067 - acc: 0.9988 - val_loss: 2.9433 - val_acc: 0.5150
Epoch 2/24
800/800 [==============================] - 4s 5ms/step - loss: 0.0109 - acc: 0.9975 - val_loss: 3.0194 - val_acc: 0.4800
Epoch 3/24
800/800 [==============================] - 4s 5ms/step - loss: 0.0183 - acc: 0.9950 - val_loss: 2.8044 - val_acc: 0.5200
Epoch 4/24
800/800 [==============================] - 4s 5ms/step - loss: 0.0082 - acc: 0.9988 - val_loss: 2.8603 - val_acc: 0.5000
Epoch 5/24
800/800 [==============================] - 4s 4ms/step - loss: 0.0068 - acc: 0.9975 - val_loss: 2.9132 - val_acc: 0.5050
Epoch 6/24
800/800 [==============================] - 4s 5ms/step - loss: 0.0050 - acc: 0.9988 - val_loss: 3.1791 - val_acc: 0.4950
Epoch 7/24
800/800 [==============================] - 4s 4ms/step - loss: 0.0064 - acc: 0.9988 - val_loss: 3.3866 - val_acc: 0.4550
Epoch 8/24
800/800 [==========

KeyboardInterrupt: 

In [39]:
score = np.zeros((2,))
for i in range(x_test.shape[0]):
    cur = model.evaluate(x_test[i:i+1], y_test[i:i+1], verbose=0)
    for idx, val in enumerate(cur):
        score[idx] += val
score /= x_test.shape[0]
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.9051771496434515
Test accuracy: 0.5


In [ ]:
score = np.zeros((2,))
for i in range(x_test.shape[0]):
    cur = model.evaluate(x_test[i:i+1], y_test[i:i+1], verbose=0)
    for idx, val in enumerate(cur):
        score[idx] += val
score /= x_test.shape[0]
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
score = np.zeros((2,))
for i in range(x_test.shape[0]):
    cur = model.evaluate(x_test[i:i+1], y_test[i:i+1], verbose=0)
    for idx, val in enumerate(cur):
        score[idx] += val
score /= x_test.shape[0]
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
score = np.zeros((2,))
for i in range(x_test.shape[0]):
    cur = model.evaluate(x_test[i:i+1], y_test[i:i+1], verbose=0)
    for idx, val in enumerate(cur):
        score[idx] += val
score /= x_test.shape[0]
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
score = np.zeros((2,))
for i in range(x_test.shape[0]):
    cur = model.evaluate(x_test[i:i+1], y_test[i:i+1], verbose=0)
    for idx, val in enumerate(cur):
        score[idx] += val
score /= x_test.shape[0]
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#0.465
model = Sequential()
model.add(Conv2D(16, kernel_size=4, strides=1, activation='relu', input_shape=input_shape))
model.add(Conv2D(16, kernel_size=4, strides=2, activation='relu'))
model.add(Dropout(0.3))
model.add(Conv2D(32, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(32, kernel_size=4, strides=2, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_OF_GENRES, activation='softmax'))

model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
#0.485
model = Sequential()
model.add(Conv2D(16, kernel_size=2, strides=1, activation='relu', input_shape=input_shape))
model.add(Conv2D(16, kernel_size=2, strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(32, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(32, kernel_size=4, strides=2, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_OF_GENRES, activation='softmax'))

model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# Test loss: 3.1804552129786376
# Test accuracy: 0.5
model = Sequential()
model.add(Conv2D(16, kernel_size=2, strides=1, activation='relu', input_shape=input_shape))
model.add(Conv2D(16, kernel_size=2, strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(32, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(32, kernel_size=4, strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_OF_GENRES, activation='softmax'))

model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=["accuracy"])